In [1]:
import pandas as pd
import plotly.express as px
import json
import re


#################################################################
# Stupid scoring
#################################################################
def score_function(row, sum_data):
    return int(row.name) + 1
    
##################################################################
#Process the word list for basic data
##################################################################
def anyalyze_word_list(word_list):
    data = {}
    for word in word_list:
        letters = list(word)
        for l in letters:
            ld = data.get(l, {'letter': l, 'count': 0})
            ld['count'] += 1
            data[l] = ld

    df = pd.DataFrame.from_dict(data, orient='index').sort_values('count').reset_index()
    
    sum_data = df['count'].sum()
    df['weight']=df['count']/sum_data
    
    df['score']=df.apply(score_function, axis=1, sum_data=sum_data)
    df=df.set_index('index')
    #print(df)
    
    #fig = px.bar(df, x='letter', y='count')
    #fig.show()
    
    word_score_data = {}
    for word in word_list:
        word_score = 0
        letters = list(word)
        for l in letters:
            df_data = df.loc[l]
            word_score += df_data.weight
        word_score_data[word] = {'word': word, 'score': word_score}
    word_df = pd.DataFrame.from_dict(word_score_data, orient='index').sort_values('score', ascending=False).reset_index()
    return word_df
    

In [2]:
###################################################
#Process the word list to based on all given
#conditions
###################################################
def validate_word(word):
    word_array = [char for char in word]
    word_set = set(word_array)
    if len(word_set) != 5:
        return False
    return True

def all_contains(contains, word):
    result = all([e in word for e in contains])
    return result

def any_contains(contains, word):
    result =  any([e in word for e in contains])
    return result

def regex_match(rex, word):
    return True if re.match(rex, word) is not None else False

def check_exclude_position(exclude, word):
    letters = list(word)
    for i in range(0, len(letters)):
        l = letters[i]
        x = exclude[i]
        if l in x:
            return False
    return True

def process(word_list, exclude, contains, rex, exclude_position):
    match = [word for word in word_list if not any_contains(exclude, word)]
    match = [word for word in match if all_contains(contains, word)]
    match = [word for word in match if regex_match(rex, word)]
    match = [word for word in match if check_exclude_position(exclude_position, word)]
    return match
    
        

In [3]:
####################################################
# Prepare our data
####################################################
results = []
with open('5_letter_words.json', 'r') as in_file:
    results = json.load(in_file)

word_df = anyalyze_word_list(results)

In [6]:
print("Guess 1")
exclude=list("ASETGH".upper())
contains=list("I") #Contains list. Should be seperate from the rex below
rex = "R...." #Used to match positions
exclude_position=['', 'GHTAISE', 'GHTAISE', 'GHTASE', 'GHTASE'] #Exclude positions

first_results = list(set(process(results, exclude, contains, rex, exclude_position)))
first_results.sort()

#Get results from DF, sorted by distrobution
df_results = word_df.loc[word_df['word'].isin(first_results)]
print(df_results.head(50))


Guess 1
      index   word     score
5745  ROBIN  ROBIN  0.275710
6140  RUNIC  RUNIC  0.271857
6145  RUNDI  RUNDI  0.271798
6559  ROMIC  ROMIC  0.266981
7234  RUBIN  RUBIN  0.259314
9039  RUVID  RUVID  0.230886
